In [1]:
epochs, batch, lr = 100, 32, 1e-4
num_clients, in_channels, num_classes = 6, 1, 10
Dataset, Model, Prefix = 'MNIST', 'CNN_3', 'Parallel'
Gdrive = '/content/drive/MyDrive/Colab Notebooks/SplitLearning3'
import torch; torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os; os.chdir(Gdrive)
%run library/Dataset.ipynb
%run library/Network.ipynb
%run library/Training.ipynb
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
def train_parallel(clients, testset, epochs=epochs, model=Model, batch=batch, lr=lr):
  in_channels, num_classes, tester = loadTester(testset)
  models = [globals()[model](in_channels, num_classes) for c in clients]
  model0 = SplitNN([Client(m.client) for m in models[:int(len(clients)/2)]], Server(models[0].server))
  model1 = SplitNN([Client(m.client) for m in models[int(len(clients)/2):]], Server(models[1].server))
  model0.initialize(clients[:int(len(clients)/2)], tester, epochs, batch, lr)
  model1.initialize(clients[int(len(clients)/2):], tester, epochs, batch, lr)

  for epoch in tqdm(range(epochs)):
    model0.train_network(epoch, private=True, sequence=False, federate=False)
    model1.train_network(epoch, private=True, sequence=False, federate=False)
    server0, server1 = model0.server.state_dict(), model1.server.state_dict()
    for key in server0:
      server0[key] += server1[key]
      if server0[key].type().split('.')[-1] == 'LongTensor':
        torch.div(server0[key], 2, rounding_mode='floor')
      else: server0[key] /= 2
    model1.server.load_state_dict(server0)
    model0.evaluate(), model1.evaluate()
    plot_progress(model0.clients + model1.clients, locals())
  return model0, model1

In [ ]:
Dataset = 'Fashion'
dataset, testset = loadDataset(Dataset)

In [15]:
Model = 'VGG_7'
clients = torch.ones((num_clients,))
clients = ratio_split(dataset, clients / clients.sum())
saveAccuracies(Gdrive, sum([m.clients for m in train_parallel(clients, (testset,1,10), model=Model)], []), 'IID-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/Fashion-VGG_7-Parallel-6-ImB-Prv.acc


In [16]:
clients = np.array([1, 3, 9, 19, 30, 38])
clients = ratio_split(dataset, clients / clients.sum())
saveAccuracies(Gdrive, sum([m.clients for m in train_parallel(clients, (testset,1,10), model=Model)], []), 'ImB-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/Fashion-VGG_7-Parallel-6-ImB-Prv.acc


In [ ]:
Dataset = 'CIFAR'
dataset, testset = loadDataset(Dataset)

In [19]:
Model = 'VGG_11'
clients = torch.ones((num_clients,))
clients = ratio_split(dataset, clients / clients.sum())
saveAccuracies(Gdrive, sum([m.clients for m in train_parallel(clients, (testset,3,10), model=Model)], []), 'IID-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Parallel-6-IID-Prv.acc


In [20]:
clients = np.array([1, 3, 9, 19, 30, 38])
clients = ratio_split(dataset, clients / clients.sum())
saveAccuracies(Gdrive, sum([m.clients for m in train_parallel(clients, (testset,3,10), model=Model)], []), 'ImB-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Parallel-6-ImB-Prv.acc
